In [1]:
import numpy as np
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.regularizers import l1,l2
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D, Conv1D, MaxPooling1D,Flatten
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
data = pd.read_csv('data.csv')
data.head()

,text,sentiment_score,sentiment
0,johnconyersjr alfranken why have you guys no...,0.2575,loosely positive
1,women have been talking about this crap the en...,-0.4588,loosely negative
2,bettemidler please speak to this sexual assaul...,-0.3612,loosely negative
3,we cant keep turning a blind eye and pretend t...,0.2163,loosely positive
4,jayz is saying what ive been saying djts jobis...,-0.6249,strongly negative


In [3]:
print(data[ data['sentiment'] == 'strongly positive'].shape)
print(data[ data['sentiment'] == 'strongly negative'].shape)
print(data[ data['sentiment'] == 'loosely positive'].shape)
print(data[ data['sentiment'] == 'loosely negative'].shape)
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

(66246, 3)
(70952, 3)
(67601, 3)
(73966, 3)


In [4]:
embed_dim = 128

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Conv1D(32, kernel_size=5, activation='relu',padding='same',bias_regularizer=l2(0.1)))
model.add(MaxPooling1D(pool_size=2, strides=2))
model.add(Conv1D(64, 5, activation='relu', padding='same', bias_regularizer=l2(0.1)))
model.add(MaxPooling1D(pool_size=2,stride=2))
model.add(Flatten())
model.add(Dense(1000, activation='relu',bias_regularizer=l2(0.1)))
model.add(Dense(4, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 54, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 54, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 54, 32)            20512     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 27, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 27, 64)            10304     
_________________________________________________________________
max_pooling1d_2 (MaxP

/var/300GB_DRIVE/ML_Project/ML/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2, strides=2)`
  if __name__ == '__main__':


In [5]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.5, random_state = 100)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
#print(Y)

(139382, 54) (139382, 4)
(139383, 54) (139383, 4)


In [6]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 40, batch_size=batch_size, verbose = 2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/40
 - 103s - loss: 0.5347 - acc: 0.7814
Epoch 2/40
 - 101s - loss: 0.4015 - acc: 0.8419
Epoch 3/40
 - 99s - loss: 0.3630 - acc: 0.8575
Epoch 4/40
 - 100s - loss: 0.3393 - acc: 0.8672
Epoch 5/40
 - 100s - loss: 0.3218 - acc: 0.8745
Epoch 6/40
 - 98s - loss: 0.3081 - acc: 0.8798
Epoch 7/40
 - 97s - loss: 0.2950 - acc: 0.8847
Epoch 8/40
 - 97s - loss: 0.2853 - acc: 0.8888
Epoch 9/40
 - 97s - loss: 0.2772 - acc: 0.8921
Epoch 10/40
 - 105s - loss: 0.2699 - acc: 0.8948
Epoch 11/40
 - 102s - loss: 0.2624 - acc: 0.8978
Epoch 12/40
 - 97s - loss: 0.2606 - acc: 0.8989
Epoch 13/40
 - 97s - loss: 0.2527 - acc: 0.9023
Epoch 14/40
 - 97s - loss: 0.2499 - acc: 0.9032
Epoch 15/40
 - 96s - loss: 0.2440 - acc: 0.9049
Epoch 16/40
 - 96s - loss: 0.2422 - acc: 0.9067
Epoch 17/40
 - 96s - loss: 0.2354 - acc: 0.9081
Epoch 18/40
 - 96s - loss: 0.2345 - acc: 0.9090
Epoch 19/40
 - 96s - loss: 0.2311 - acc: 0.9111
Epoch 20/40
 - 96s - loss: 0.2294 - acc: 0.

In [7]:
validation_size = 68000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.53
acc: 0.87


In [8]:
#Validation
loosely_neg_cnt,loosely_pos_cnt,strongly_neg_cnt,strongly_pos_cnt,loosely_neg_correct,loosely_pos_correct,strongly_neg_correct,strongly_pos_correct = 0,0,0,0,0,0,0,0

for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            loosely_neg_correct += 1
        elif np.argmax(Y_validate[x]) == 1:
            loosely_pos_correct += 1
        elif np.argmax(Y_validate[x]) == 2:
            strongly_neg_correct += 1
        elif np.argmax(Y_validate[x]) == 3:
            strongly_pos_correct += 1
            
    if np.argmax(Y_validate[x]) == 0:
        loosely_neg_cnt += 1
    elif np.argmax(Y_validate[x]) == 1:
        loosely_pos_cnt += 1
    elif np.argmax(Y_validate[x]) == 2:
        strongly_neg_cnt += 1
    elif np.argmax(Y_validate[x]) == 3:
        strongly_pos_cnt += 1

print("loosely_pos_acc", loosely_pos_correct/loosely_pos_cnt*100, "%")
print("loosely_neg_acc", loosely_neg_correct/loosely_neg_cnt*100, "%")
print("strongly_pos_acc", strongly_pos_correct/strongly_pos_cnt*100, "%")
print("strongly_neg_acc", strongly_neg_correct/strongly_neg_cnt*100, "%")

loosely_pos_acc 82.68019808033257 %
loosely_neg_acc 85.47392970181575 %
strongly_pos_acc 90.17186821368742 %
strongly_neg_acc 89.40457840954501 %


In [10]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=54, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
sentiment
if np.argmax(sentiment) == 0:
    print("Loosely Negative")
elif np.argmax(sentiment) == 1:
    print("Loosely Positive")
elif np.argmax(sentiment) == 2:
    print("Strongly Negative")
elif np.argmax(sentiment) == 3:
    print("Strongly Positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 135   5  44   8  29  29  31   5  44]]
Strongly Negative
